In [1]:
import cv2
import mediapipe as mp
import numpy as np

def eye_aspect_ratio(landmarks, left_eye_indices, right_eye_indices):
    def get_eye_ratio(eye_points):
        A = np.linalg.norm(np.array(eye_points[1]) - np.array(eye_points[5]))
        B = np.linalg.norm(np.array(eye_points[2]) - np.array(eye_points[4]))
        C = np.linalg.norm(np.array(eye_points[0]) - np.array(eye_points[3]))
        return (A + B) / (2.0 * C)
    
    left_eye = [landmarks[i] for i in left_eye_indices]
    right_eye = [landmarks[i] for i in right_eye_indices]
    
    left_ratio = get_eye_ratio(left_eye)
    right_ratio = get_eye_ratio(right_eye)
    
    return (left_ratio + right_ratio) / 2.0

mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
pose = mp_pose.Pose()

cap = cv2.VideoCapture(0)
EYE_AR_THRESHOLD = 0.25
DROWSY_FRAMES = 30
frame_counter = 0

eye_indices = {
    "left": [33, 160, 158, 133, 153, 144],
    "right": [362, 385, 387, 263, 373, 380]
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_face = face_mesh.process(frame_rgb)
    results_pose = pose.process(frame_rgb)
    
    h, w, _ = frame.shape
    landmarks = []
    
    if results_face.multi_face_landmarks:
        for face_landmarks in results_face.multi_face_landmarks:
            landmarks = [(int(pt.x * w), int(pt.y * h)) for pt in face_landmarks.landmark]
            ear = eye_aspect_ratio(landmarks, eye_indices['left'], eye_indices['right'])
            
            if ear < EYE_AR_THRESHOLD:
                frame_counter += 1
            else:
                frame_counter = 0
            
            if frame_counter >= DROWSY_FRAMES:
                cv2.putText(frame, "Drowsy!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    if results_pose.pose_landmarks:
        nose = results_pose.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
        if nose.visibility > 0.5:
            nose_y = nose.y * h
            if nose_y > h * 0.7:
                cv2.putText(frame, "Head Down!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    
    cv2.imshow("Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\10\anaconda3\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
